# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

**Ans.** As mentioned in the first sentence of the second paragraph, there were **17 campaigns**.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
import pandas as pd

In [2]:
# df = pd.read_csv('data/bank-additional/bank-additional-full.csv', sep = ';')
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [3]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```

**Ans.** Check the data time of every column:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

And as you saw the part of the data set (`df.head()`) in *Problem 2*, some fields have labels such as *"married"*, binary fields such as *"yes"* or *"no"*, and even ternary fields with the *"unknown"* label.

For the label features, you need to replace the column by dummy features. And for the binary or ternary features, replace those by values between 0.0 for *"no"* and 1.0 for *"yes"*. Replace *"unknown"* entries by the mean of the column.

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

**Ans.** Predict how likely a new client starts subscribing a term deposit, based on the input parameters: his or her age, job status, marital status, and so on.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

**Ans.** Construct a new data frame `df_optimized` by copying and modifying the original `df` data frame:

* Columns with floating point entries: simply copy the column in `df` to `df_optimized`.

* Columns with integer entries: convert to floating point.

* Columns with "yes", "no", or "unknown" entries: convert these entries to 1.0, 0.0, or `np.nan` respectively. Later, replace the null values by the mean value of the column.

* Columns with all other string entries: apply the one-hot encoding and replace those columns by ones of dummy features.

In [5]:
df_optimized = pd.DataFrame()
df_optimized = df.select_dtypes(include='float64')
df_optimized.head()

,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,1.1,93.994,-36.4,4.857,5191.0
1,1.1,93.994,-36.4,4.857,5191.0
2,1.1,93.994,-36.4,4.857,5191.0
3,1.1,93.994,-36.4,4.857,5191.0
4,1.1,93.994,-36.4,4.857,5191.0


In [6]:
df_optimized = df.select_dtypes(include='int64').astype('float64')
df_optimized.head()

,age,duration,campaign,pdays,previous
0,56.0,261.0,1.0,999.0,0.0
1,57.0,149.0,1.0,999.0,0.0
2,37.0,226.0,1.0,999.0,0.0
3,40.0,151.0,1.0,999.0,0.0
4,56.0,307.0,1.0,999.0,0.0


In [7]:
import numpy as np

In [8]:
colname_ternary = ['default','housing','loan','y']

for colname in colname_ternary:
    df_optimized[colname] = df[colname].map(
        {'yes': 1.0, 'no': 0.0, 'unknown': np.nan}
        )
df_optimized.fillna(df_optimized.mean(), inplace=True)
df_optimized.head()

,age,duration,campaign,pdays,previous,default,housing,loan,y
0,56.0,261.0,1.0,999.0,0.0,0.000000,0.0,0.0,0.0
1,57.0,149.0,1.0,999.0,0.0,0.000092,0.0,0.0,0.0
2,37.0,226.0,1.0,999.0,0.0,0.000000,1.0,0.0,0.0
3,40.0,151.0,1.0,999.0,0.0,0.000000,0.0,0.0,0.0
4,56.0,307.0,1.0,999.0,0.0,0.000000,0.0,1.0,0.0


Encode the object values to dummy feature integers by using the one-hot encoding method:

In [9]:
df_dummies = pd.get_dummies(
    df.select_dtypes(include=['object']).drop(columns=colname_ternary),
    dtype=int
    )

df_optimized = pd.concat([df_optimized,df_dummies], axis=1)

del df_dummies
df_optimized

,age,duration,campaign,pdays,previous,default,housing,loan,y,job_admin.,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,56.0,261.0,1.0,999.0,0.0,0.000000,0.0,0.0,0.0,0,...,0,0,0,1,0,0,0,0,1,0
1,57.0,149.0,1.0,999.0,0.0,0.000092,0.0,0.0,0.0,0,...,0,0,0,1,0,0,0,0,1,0
2,37.0,226.0,1.0,999.0,0.0,0.000000,1.0,0.0,0.0,0,...,0,0,0,1,0,0,0,0,1,0
3,40.0,151.0,1.0,999.0,0.0,0.000000,0.0,0.0,0.0,1,...,0,0,0,1,0,0,0,0,1,0
4,56.0,307.0,1.0,999.0,0.0,0.000000,0.0,1.0,0.0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73.0,334.0,1.0,999.0,0.0,0.000000,1.0,0.0,1.0,0,...,0,0,1,0,0,0,0,0,1,0
41184,46.0,383.0,1.0,999.0,0.0,0.000000,0.0,0.0,0.0,0,...,0,0,1,0,0,0,0,0,1,0
41185,56.0,189.0,2.0,999.0,0.0,0.000000,1.0,0.0,0.0,0,...,0,0,1,0,0,0,0,0,1,0
41186,44.0,442.0,1.0,999.0,0.0,0.000000,0.0,0.0,1.0,0,...,0,0,1,0,0,0,0,0,1,0


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

**Ans.** Apply the `train_test_split` method of scikit-learn to the data frame `df_optimized`. Shaffle all rows in  with the `random_state=42`, and split these rows vertically into the 75% train set and the 25% test set. Also split those rows horizontally into input matrices, `X_train` and `X_test`, and output column vectors, `y_train` and `y_test`.

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# df.iloc[:,-1]   # Slect the last column.
# df.iloc[:,:-1]  # Select all columns but the last one.

X_train, X_test, y_train, y_test = train_test_split(
    df_optimized.drop(columns=['y']), df_optimized['y'], 
    test_size=0.25,  # Default
    random_state = 42
)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

**Ans.** The `y` column in the original data set contains either 1.0 for *"yes"* or 0.0 for *"no"*. Check what is in the `y` column of the training data set:

In [12]:
y_train.value_counts()

y
0.0    27404
1.0     3487
Name: count, dtype: int64

As you can see in the output, the majority of the `y` field in the training set is 0.0 for *"no"*. **Define the baseline as all 0.0**.

In [13]:
baseline = pd.Series(['0.0'] * len(y_test))
baseline

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
10292    0.0
10293    0.0
10294    0.0
10295    0.0
10296    0.0
Length: 10297, dtype: object

Check the errors of the baseline:

In [14]:
print("Baseline (all no / all samples) - Accuracy")
print(f"Train Accuracy: {(y_train == 0.0).mean() * 100: .2f}%")
print(f"Test Accuracy:  {(y_test  == 0.0).mean() * 100: .2f}%")

Baseline (all no / all samples) - Accuracy
Train Accuracy:  88.71%
Test Accuracy:   88.80%


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
# from sklearn.impute import SimpleImputer

In [16]:
# discussion_16.1 - Task 2

linreg_pipe = Pipeline([
#    ('poly_features', PolynomialFeatures(
#        degree = 3, include_bias = False)
#        ),
    ('scaler',   StandardScaler()),
#   ('selector', SelectFromModel(Lasso())),
    ('linreg',   LogisticRegression())
])

linreg_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('linreg', LogisticRegression())])

### Problem 9: Score the Model

What is the accuracy of your model?

**Ans.** For the train data set and the test data set, simply divide the number of *"no"* by the total number of rows in the output column `y` .

In [17]:
print("Linear Regerssion")

pred_train = linreg_pipe.predict(X_train)
print(f"Train Accuracy: {(y_train == pred_train).mean() * 100: .2f}%")

pred_test  = linreg_pipe.predict(X_test)
print(f"Test Accuracy:  {(y_test  == pred_test ).mean() * 100: .2f}%")

Linear Regerssion
Train Accuracy:  90.73%
Test Accuracy:   90.76%


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

**Ans.** Reuse snippets in Discussion 16.1.

In [18]:
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [19]:
models = {
    'Logistic Regression': LogisticRegression(),
    'KNN':                 KNeighborsClassifier(),
    'Decision Tree':       DecisionTreeClassifier(),
    'SVM':                 SVC()
    }

results = []

for model_name, model in models.items():
    pipe = Pipeline([
        ('scaler',   StandardScaler()),
        (model_name, model)
    ])
    
    start_time = time.time()
    pipe.fit(X_train, y_train)
    fit_time = time.time()
    
    print(model_name)

    duration_train = fit_time - start_time
    print(f'Train time    : {duration_train: .2f} sec.')
    
    pred_train = pipe.predict(X_train)
    accuracy_train = (y_train == pred_train).mean()
    print(f"Train Accuracy: {accuracy_train * 100: .2f}%")
    pred_test  = pipe.predict(X_test)
    accuracy_test  = (y_test  == pred_test ).mean()
    print(f"Test Accuracy:  {accuracy_test  * 100: .2f}%")

    results.append([model_name, duration_train, accuracy_train, accuracy_test])

Logistic Regression
Train time    :  0.03 sec.
Train Accuracy:  90.73%
Test Accuracy:   90.76%
KNN
Train time    :  0.01 sec.
Train Accuracy:  91.80%
Test Accuracy:   89.78%
Decision Tree
Train time    :  0.13 sec.
Train Accuracy:  100.00%
Test Accuracy:   87.47%
SVM
Train time    :  7.28 sec.
Train Accuracy:  92.03%
Test Accuracy:   90.64%


In [20]:
pd.DataFrame(
    [['Baseline', np.nan, (y_train == 0.0).mean(), (y_test  == 0.0).mean()]]
    + results,
    columns=['Model','Train Time [s]', 'Train Accuracy', 'Test Accuracy']
    ).round(4)

,Model,Train Time [s],Train Accuracy,Test Accuracy
0,Baseline,NaN,0.8871,0.8880
1,Logistic Regression,0.0281,0.9073,0.9076
2,KNN,0.0095,0.9180,0.8978
3,Decision Tree,0.1283,1.0000,0.8747
4,SVM,7.2822,0.9203,0.9064


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

**More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?**

**Ans.** The original data set did not contain the "gender" field. From a data science standpoint, it is usually meaningful to add more features and see how they affect the prediction accuracy. In reality, however, collecting, storing, and utilizing this kind of feature may connect to social inequality, and the machine learning can even produce such an unfavorable output as *"single women are not likely to use our service"*.

**Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.**

**Ans.** Repeat evaluation with those three models and three different parameters for each:

In [21]:
models = {
    'LogisticRegression(C= 0.1)': LogisticRegression(C= 0.1),
    'LogisticRegression(C= 1.0)': LogisticRegression(C= 1.0),
    'LogisticRegression(C=10.0)': LogisticRegression(C=10.0),

    'KNeighborsClassifier(n_neighbors=3)': KNeighborsClassifier(n_neighbors=3),
    'KNeighborsClassifier(n_neighbors=5)': KNeighborsClassifier(n_neighbors=5),
    'KNeighborsClassifier(n_neighbors=7)': KNeighborsClassifier(n_neighbors=7),

    'DecisionTreeClassifier(max_depth= 5)': 
        DecisionTreeClassifier(max_depth= 5),
    'DecisionTreeClassifier(max_depth=10)': 
        DecisionTreeClassifier(max_depth=10),
    'DecisionTreeClassifier(max_depth=15)': 
        DecisionTreeClassifier(max_depth=15),

    'SVC(C= 0.1, kernel=\'rbf\')': SVC(C= 0.1, kernel='rbf'),
    'SVC(C= 1.0, kernel=\'rbf\')': SVC(C= 1.0, kernel='rbf'),
    'SVC(C=10.0, kernel=\'rbf\')': SVC(C=10.0, kernel='rbf'),
    'SVC(C= 0.1, kernel=\'linear\')': SVC(C= 0.1, kernel='linear'),
    'SVC(C= 1.0, kernel=\'linear\')': SVC(C= 1.0, kernel='linear'),
#   'SVC(C=10.0, kernel=\'linear\')': SVC(C=10.0, kernel='linear')
    }

results = []

for model_name, model in models.items():
    pipe = Pipeline([
        ('scaler',   StandardScaler()),
        (model_name, model)
    ])
    
    start_time = time.time()
    pipe.fit(X_train, y_train)
    fit_time = time.time()
    
    print(model_name)

    duration_train = fit_time - start_time
    print(f'Train time    : {duration_train: .2f} sec.')
    
    pred_train = pipe.predict(X_train)
    accuracy_train = (y_train == pred_train).mean()
    print(f"Train Accuracy: {accuracy_train * 100: .2f}%")
    pred_test  = pipe.predict(X_test)
    accuracy_test  = (y_test  == pred_test ).mean()
    print(f"Test Accuracy:  {accuracy_test  * 100: .2f}%")

    results.append([model_name, duration_train, accuracy_train, accuracy_test])

LogisticRegression(C= 0.1)
Train time    :  0.07 sec.
Train Accuracy:  90.71%
Test Accuracy:   90.76%
LogisticRegression(C= 1.0)
Train time    :  0.03 sec.
Train Accuracy:  90.73%
Test Accuracy:   90.76%
LogisticRegression(C=10.0)
Train time    :  0.03 sec.
Train Accuracy:  90.73%
Test Accuracy:   90.77%
KNeighborsClassifier(n_neighbors=3)
Train time    :  0.01 sec.
Train Accuracy:  93.10%
Test Accuracy:   89.22%
KNeighborsClassifier(n_neighbors=5)
Train time    :  0.01 sec.
Train Accuracy:  91.80%
Test Accuracy:   89.78%
KNeighborsClassifier(n_neighbors=7)
Train time    :  0.01 sec.
Train Accuracy:  91.14%
Test Accuracy:   90.05%
DecisionTreeClassifier(max_depth= 5)
Train time    :  0.04 sec.
Train Accuracy:  91.12%
Test Accuracy:   91.08%
DecisionTreeClassifier(max_depth=10)
Train time    :  0.08 sec.
Train Accuracy:  93.09%
Test Accuracy:   90.70%
DecisionTreeClassifier(max_depth=15)
Train time    :  0.11 sec.
Train Accuracy:  95.90%
Test Accuracy:   89.48%
SVC(C= 0.1, kernel='rbf')

In [22]:
pd.DataFrame(
    [['Baseline', np.nan, (y_train == 0.0).mean(), (y_test  == 0.0).mean()]]
    + results,
    columns=['Model','Train Time [s]', 'Train Accuracy', 'Test Accuracy']
    ).round(4)

,Model,Train Time [s],Train Accuracy,Test Accuracy
0,Baseline,NaN,0.8871,0.8880
1,LogisticRegression(C= 0.1),0.0746,0.9071,0.9076
2,LogisticRegression(C= 1.0),0.0265,0.9073,0.9076
3,LogisticRegression(C=10.0),0.0268,0.9073,0.9077
4,KNeighborsClassifier(n_neighbors=3),0.0104,0.9310,0.8922
5,KNeighborsClassifier(n_neighbors=5),0.0107,0.9180,0.8978
6,KNeighborsClassifier(n_neighbors=7),0.0100,0.9114,0.9005
7,DecisionTreeClassifier(max_depth= 5),0.0434,0.9112,0.9108
8,DecisionTreeClassifier(max_depth=10),0.0760,0.9309,0.9070
9,DecisionTreeClassifier(max_depth=15),0.1062,0.9590,0.8948


As you can see in the "Test Accuracy" column, all models exceeded the baseline in terms of the accuracy for unseen data. The decision tree of max depth 5 marked the best accuracy: 91.08% vs. 88.80%. In the second place, the logistic regression with the regularization strength $C = 10.0$ or $\lambda = 0.1$ achieved the accuracy of 90.77%, and it took 38% less time than the decision tree. The K-nearest neighbors with 7 neighbors marked mere 90.05% but it took 77% less time than the decision tree. The SVM with the radical basis function kernel, also known as the Gaussian kernel, performed poorly both in terms of the accuracy and train time.


**Adjust your performance metric**

**Ans.** No need to adjust any performance metric. &#9632;